In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
areas=avisos['nombre_area'].value_counts()
avisos['nombre_area']=avisos['nombre_area'].replace(areas.index, areas.values)

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral','nombre_area']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')
avisos_no_one_hot['nombre_area']=avisos_no_one_hot['nombre_area'].astype('category')
avisos_no_one_hot=avisos_no_one_hot.rename(columns={'nombre_area': 'nombre_area_aux'})

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [11]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [12]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [13]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501935


In [14]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad
9436459,1112401866,qewxXkL,0,43.0,False,True,False,False,False,False,...,False,False,True,False,Full-time,Otro,Ventas,1.0,NaN,NaN
9007971,1112389830,YWRX11,0,33.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Mantenimiento,NaN,NaN,NaN
5902125,1112359974,vVDwAmx,0,20.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Almacén / Depósito / Expedición,3.0,NaN,NaN
3740759,1112303993,owEzVN0,1,22.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Producción,NaN,NaN,NaN
11038620,1112320610,ZDPkpLR,0,24.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Finanzas,3.0,NaN,NaN
16021679,1111409681,1QP6NZB,0,31.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Calidad,24.0,NaN,NaN
5941313,1112463123,xkvKEkP,0,25.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Administración,NaN,NaN,NaN
10968930,1112390462,LNw5vGp,1,32.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Logística,659.0,NaN,NaN
2270049,1112282953,xkdqlNz,1,22.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Administración,2.0,NaN,NaN
5300587,1112223976,ZOpvD5,0,35.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Mineria/Petroleo/Gas,15.0,NaN,NaN


In [15]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad
547963,1112315349,DrJ0WWP,1,19.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Recepcionista,47.0,0.0,0.000000
15849689,1112085369,8K4jaW,0,27.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Arquitectura,12.0,0.0,0.000000
13484664,1112347328,ekNeXpE,1,24.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Recepcionista,9.0,0.0,-0.900000
13659518,1112158000,6rZNQ9l,0,27.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Atención al Cliente,0.0,0.0,0.000000
11039668,1112336464,2zPDzXo,0,32.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Administración,8.0,0.0,0.000000
13847021,1112264031,Drzw98Z,0,29.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,0.0,0.0,0.000000
2588864,1112309001,JBxLQYk,1,19.0,False,True,False,False,False,False,...,False,True,False,False,Full-time,Junior,Ventas,25.0,0.0,0.000000
8210183,1112469138,JBroLlb,1,30.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Tesorería,116.0,0.0,-0.991453
3071147,1112300796,5mqJZKG,0,18.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Administración,13.0,0.0,0.000000
5349771,1112366184,bOJr5BE,1,33.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Almacén / Depósito / Expedición,8.0,0.0,0.000000


In [16]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area_aux'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area_aux,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,1
2,0z5Dmrd,Administración de Personal,1
3,0z5Dmrd,Almacén / Depósito / Expedición,2
4,0z5Dmrd,Atención al Cliente,1


In [17]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
1,0z5Dmrd,Junior,2
2,0z5Dmrd,Otro,1
3,0z5Dmrd,Senior / Semi-Senior,18
4,0z5JW1r,Gerencia / Alta Gerencia / Dirección,2


In [18]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,21
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,27
3,0z5JW1r,Teletrabajo,1
4,0z5VvGv,Full-time,53


In [19]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
2797610,1112481141,aMBdpE,0,40.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Producción,7.0,0.0,0.000000,0,29,23
8364724,1112486908,MVelWqK,0,58.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Ventas,3.0,0.0,0.000000,1,18,11
7879910,1112466900,akx4LoN,1,21.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Comercial,35.0,0.0,0.000000,43,276,203
13627306,1112243535,1l09jz,0,38.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Programación,0.0,0.0,0.000000,0,19,15
11546288,1112418929,5WLkrw,1,32.0,True,False,False,False,False,False,...,False,Full-time,Junior,Compras Internacionales/Importación,294.0,0.0,-0.996610,6,257,60
11188165,1112273420,DrXMXMX,0,18.0,True,False,False,False,False,False,...,False,Full-time,Junior,Impuestos,10.0,0.0,0.000000,0,14,5
11466556,1112403117,qeYzqbR,0,30.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Programación,20.0,0.0,0.000000,1,18,16
6850587,1112475003,QNEEL8l,0,24.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Ingeniería Oficina Técnica / Proyecto,29.0,0.0,0.000000,0,17,12
6998079,1112457348,6a6BKx,1,34.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Mantenimiento y Limpieza,177.0,0.0,-0.994382,6,142,92
15459411,1112411664,ZZAWb8,0,46.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Farmacéutica,59.0,0.0,0.000000,0,32,19


In [20]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))

Train:  15281806 Test:  804306


In [21]:
features=['edad','sexo_FEM', 'sexo_MASC','Doctorado_Abandonado',
'Doctorado_En Curso', 'Doctorado_Graduado', 'Master_Abandonado',
'Master_En Curso', 'Master_Graduado', 'Posgrado_Abandonado',
'Posgrado_En Curso', 'Posgrado_Graduado', 'Secundario_Abandonado',
'Secundario_En Curso', 'Secundario_Graduado', 'Terciario/Técnico_Abandonado',
'Terciario/Técnico_En Curso', 'Terciario/Técnico_Graduado', 'Universitario_Abandonado',
'Universitario_En Curso', 'Universitario_Graduado', 'pide_hombre',
'pide_mujer', 'ingles', 'experiencia', 'paquete_office', 'liderazgo', 'secundario',
'viajar', 'empresa_importante', 'capacitacion', 'remuneracion_pretendida', 'graduados',
'estudiantes', 'conocimientos_especificos',
'tipo_de_trabajo_Fines de Semana', 'vistas_postulante',
'vistas_al_aviso', 'cantidad', 'postulaciones_misma_area', 'postulaciones_mismo_tipo', 
'postulaciones_mismo_nivel']

In [22]:
x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

In [23]:
postulaciones=0
c=0
gc.collect()

185

In [ ]:
c=MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto',epsilon=1e-08, hidden_layer_sizes=(len(features), 5), learning_rate='constant',learning_rate_init=0.001, max_iter=200,nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=True,warm_start=False)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/NN.pkl")
dt=c.fit(x_train,y_train)

Iteration 1, loss = 0.33158821
Iteration 2, loss = 0.31235394
Iteration 3, loss = 0.30960502
Iteration 4, loss = 0.30627342
Iteration 5, loss = 0.30341324
Iteration 6, loss = 0.30394357
Iteration 7, loss = 0.30380121
Iteration 8, loss = 0.30315766


In [25]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9417914447968304


In [26]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/NN v3.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/NN v3.pkl']

In [27]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
60981,8948,1111802113,52L6ap,17.0,False,True,False,False,False,False,...,False,Full-time,Junior,Administración de Personal,2.0,0.0,0.00000,0.0,17.0,3.0
99163,99097,1112509474,ZWmel5,30.0,True,False,False,False,False,False,...,True,Part-time,Senior / Semi-Senior,Call Center,1.0,0.0,0.00000,2.0,1.0,11.0
51245,49411,1112437890,VNr5Ov2,20.0,False,True,False,False,False,False,...,False,Full-time,-,Abastecimiento,1.0,0.0,0.00000,1.0,28.0,1.0
78243,24880,1112325061,YVqoEZ,31.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Logística,4.0,0.0,0.00000,1.0,21.0,14.0
92505,91530,1112471849,ErXro9,28.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Administración,11.0,0.0,0.00000,3.0,21.0,10.0
3973,329,1788990,aR3lZW,48.0,False,True,False,False,False,False,...,False,Full-time,Otro,Calidad,4.0,0.0,0.00000,2.0,21.0,3.0
50494,37937,1112414534,QNlAllV,28.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Administración,48.0,0.0,0.00000,11.0,45.0,36.0
20637,53911,1112445548,0zdrw58,27.0,True,False,False,False,False,False,...,False,Full-time,Junior,Administración de Personal,160.0,3.0,0.01875,5.0,88.0,16.0
41552,40605,1112420016,RzrqY91,29.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Cadetería,3.0,0.0,0.00000,0.0,17.0,11.0
67457,13528,1112043613,NPR5xB,42.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,Seguridad,27.0,0.0,0.00000,1.0,39.0,26.0


In [28]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.84854876, 0.15145124],
       [0.4028937 , 0.5971063 ],
       [0.9225451 , 0.0774549 ],
       ...,
       [0.98284606, 0.01715394],
       [0.95993151, 0.04006849],
       [0.99317916, 0.00682084]])

In [29]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [30]:
prediccion['sepostulo']=y_final2

In [31]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,Full-time,Jefe / Supervisor / Responsable,Comercial,33.0,0.0,0.0,1.0,20.0,1.0,0.151451
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,Full-time,Jefe / Supervisor / Responsable,Comercial,8.0,0.0,0.0,6.0,60.0,0.0,0.597106
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,Full-time,Jefe / Supervisor / Responsable,Comercial,12.0,0.0,0.0,3.0,15.0,1.0,0.077455
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,Full-time,Otro,Tecnologia / Sistemas,0.0,0.0,0.0,2.0,17.0,1.0,0.085818
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,Full-time,Otro,Tecnologia / Sistemas,3.0,0.0,0.0,1.0,16.0,1.0,0.027108


In [32]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.151451
1,1,0.597106
2,2,0.077455
3,3,0.085818
4,4,0.027108


In [33]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion49.csv",index=False)

In [34]:
prediccion.sepostulo.mean()

0.47497934463605335